In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Mar 17 16:19:56 2025

@author: user
"""

# =============================================================================
# 
import os
import netCDF4 as nc
import pandas as pd
import numpy as np
import numpy.ma as ma
import xarray as xr
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.style as mplstyle
import cartopy.crs as ccrs
import netCDF4 as nc
from netCDF4 import Dataset
import shapefile
import time
# =============================================================================
import warnings
warnings.filterwarnings("ignore")  # 忽略所有警告


import shapefile
# from wrf import getvar, ALL_TIMES
from datetime import datetime, timedelta
import cmaps
import sys
# 将目标模块所在的路径添加到 sys.path 中
sys.path.append(r"/home/yfdong/data/work/LF-SAM/code/Library")
from MeteoVarPlot import draw_2DVAR, draw_Bias, draw_VARtemporal, draw_Bias_subplot,draw_2DVAR_subplot ,draw_2DuvVAR ,draw_uvBias_subplot ,draw_2DuvVAR_subplot
from MeteoChartPlot import add_TickGrid
from PreprocessVar import (
    get_MONTH_abbr, get_pentad, get_letter,
    preprocess_var, read_var,Events_configs,
    Read_VarsFromVarsDict, experiments_types,return_Event_Date,
    load_geodata, SubsetDomainConfigure ,create_domain_mask,get_letter,
    get_CoordPair
)
experiment = experiments_types[0]
import matplotlib.dates as mdates
BasePath = "/home/yfdong/data/work/LF-SAM/code/Library"
DataPath = '/raid61/yfdong/data/work/LF-SAM/output/'
# 读取地形数据
LAT, LON, LandMask, DEM = load_geodata(GeoPath ="/raid61/yfdong/data/work/LF-SAM/Domain/def/geo_em.nc" ,IF_LandMask=True)

In [ ]:
Year = 2011
Read_StartDate = f'{Year}-06-01'
Read_EndDate = f'{Year}-08-31'
Timedelta = 24
var = 'SH2O_Layer1'
DateLists = pd.date_range(Read_StartDate, Read_EndDate, freq=f'{Timedelta}h')
with Dataset(os.path.join(DataPath, str(Year), 'DST', 'var_nc', f'{Year}-def_HCTRL-{var}_Daily_MEAN.nc')) as ds:
    WRF_H_var = ds.variables[var][:, :, :]

with Dataset(os.path.join(DataPath, str(Year), 'DST', 'var_nc', f'{Year}-def_WCTRL-{var}_Daily_MEAN.nc')) as ds:
    WRF_S_var = ds.variables[var][:, :, :]


In [ ]:
ObsData_raw = pd.read_hdf('/home/yfdong/data/work/SMmerge/CN/merge/code/analysis_code/DataBase/Total_merged_test_SM_withBASIN_CB_10cm_GDS_Daily.h5', key='data')
ObsData_raw['Date'] = pd.to_datetime(ObsData_raw['Date'])

In [ ]:
print(len(ObsData_raw))
ObsData_raw = ObsData_raw.drop_duplicates(subset=['Date', 'ID'])
print(len(ObsData_raw))

In [ ]:
Domain = [109, 125, 25, 38]
ObsData = ObsData_raw[(ObsData_raw['Date'] >= pd.to_datetime(Read_StartDate)) & (ObsData_raw['Date'] <= pd.to_datetime(Read_EndDate))]
ObsData = ObsData[(ObsData['Longitude'] >= Domain[0]) & (ObsData['Longitude'] <= Domain[1]) & (ObsData['Latitude'] >= Domain[2]) & (ObsData['Latitude'] <= Domain[3])]

In [ ]:
from scipy.interpolate import griddata
from tqdm import tqdm

def Grid_To_ID(src_var, src_lat, src_lon, df, df_variable_name):
    """优化版本：减少不必要的复制"""
    # 提取站点坐标
    ID_lat = df["Latitude"].values
    ID_lon = df["Longitude"].values
    
    # 将格点数据转换为平面坐标（只做一次）
    points = np.column_stack((src_lon.ravel(), src_lat.ravel()))
    values = src_var.ravel()
    
    # 执行插值
    ID_SM_interpolated = griddata(points, values, (ID_lon, ID_lat), method='nearest')
    
    # 直接返回插值结果，不复制整个DataFrame
    return ID_SM_interpolated

# 优化后的主循环
# 预计算格点坐标（避免重复计算）
points = np.column_stack((LON.ravel(), LAT.ravel()))

# 提前提取站点坐标
ID_lat = ObsData["Latitude"].values
ID_lon = ObsData["Longitude"].values

# 使用列表收集结果，最后一次性合并
WRF_H_results = []
WRF_S_results = []

for DateIndex, Date in enumerate(tqdm(DateLists, desc="Processing Dates")):
    # 提取单个时间步数据
    WRF_H_var_single = WRF_H_var[DateIndex, :, :]
    WRF_S_var_single = WRF_S_var[DateIndex, :, :]
    
    # 插值（使用预计算的坐标）
    WRF_H_interpolated = griddata(points, WRF_H_var_single.ravel(), (ID_lon, ID_lat), method='nearest')
    WRF_S_interpolated = griddata(points, WRF_S_var_single.ravel(), (ID_lon, ID_lat), method='nearest')
    
    # 创建临时DataFrame并添加到列表
    temp_H = ObsData.copy()
    temp_H['WRF-H'] = WRF_H_interpolated
    temp_H['Date'] = Date
    
    temp_S = ObsData.copy()
    temp_S['WRF-S'] = WRF_S_interpolated
    temp_S['Date'] = Date
    
    WRF_H_results.append(temp_H)
    WRF_S_results.append(temp_S)

# 一次性合并所有结果
WRF_H_var_ID_df = pd.concat(WRF_H_results, ignore_index=True)
WRF_S_var_ID_df = pd.concat(WRF_S_results, ignore_index=True)


In [ ]:
print(len(WRF_H_var_ID_df))
WRF_S_var_ID_df = WRF_S_var_ID_df.drop_duplicates(subset=['Date', 'ID'])
WRF_H_var_ID_df = WRF_H_var_ID_df.drop_duplicates(subset=['Date', 'ID'])
print(len(WRF_S_var_ID_df))

In [ ]:
WRF_S_var_ID_df = WRF_S_var_ID_df[['ID', 'Date', 'WRF-S']]
WRF_H_var_ID_df = WRF_H_var_ID_df[['ID', 'Date', 'WRF-H']]

In [ ]:
WRF_H_var_ID_df['Date'] = pd.to_datetime(WRF_H_var_ID_df['Date'])
WRF_S_var_ID_df['Date'] = pd.to_datetime(WRF_S_var_ID_df['Date'])
db_df = pd.merge(ObsData_raw, WRF_H_var_ID_df, on=['Date','ID'], how='inner')
db_df = pd.merge(db_df, WRF_S_var_ID_df, on=['Date','ID'], how='inner')
db_df = db_df.dropna()
db_df

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy.stats import pearsonr

def calculate_metrics(simulated, observed):
    """计算模拟值与观测值之间的评分指标"""
    # 移除NaN值
    mask = ~np.isnan(simulated) & ~np.isnan(observed)
    sim_valid = simulated[mask]
    obs_valid = observed[mask]
    
    if len(sim_valid) < 2:  # 至少需要2个点计算相关系数
        return np.nan, np.nan, np.nan
    
    # 计算相关系数
    corr, _ = pearsonr(sim_valid, obs_valid)
    
    # 计算均方根误差
    rmse = np.sqrt(np.mean((sim_valid - obs_valid) ** 2))
    
    # 计算偏差
    bias = np.mean(sim_valid - obs_valid)
    
    return corr, rmse, bias

def plot_SM_precipitation(ID_df, ID_Index, ID_Name, ax1):
    ID_Lat = np.nanmean(ID_df['Latitude']).round(2)
    ID_Lon = np.nanmean(ID_df['Longitude']).round(2)

    # --------- 绘制土壤湿度模拟差异柱状图（左侧坐标轴）----------------------
    ax1.bar(ID_df['Date'], (ID_df['WRF-H'] - ID_df['WRF-S']), 
            width=1, alpha=1, 
            color="#009ACD", edgecolor='black', linewidth=0.5,
            label="Diff", zorder=1)

    # ----------- 绘制土壤湿度观测（右侧坐标轴）----------------------
    ax1.set_ylim(-0.05, 0.6)
    ax1.plot(ID_df['Date'], ID_df['OBS_SM_10cm'], 
             color="#363636", 
             linewidth=2, alpha=1, label="OBS", zorder=10)
    ax1.plot(ID_df['Date'], ID_df['WRF-H'], 
             color="#00BFFF", 
             linewidth=2, alpha=1, label="WRF-H", zorder=10)
    ax1.plot(ID_df['Date'], ID_df['WRF-S'], 
             color="#FF7256", 
             linewidth=2, alpha=1, label="WRF-S", zorder=10)

    # 时间标签设置
    ax1.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
    ax1.xaxis.set_major_formatter(mdates.DateFormatter("%b"))
    ax1.minorticks_on()
    ax1.tick_params(axis="both", which="major", direction="in", width=0.9, length=5, color='#363636')
    ax1.grid(True, which="major", linestyle="--", color="gray", linewidth=0.75, alpha=0.3)
    
    subTitle = f"({get_letter(ID_Index)}) ID: {ID_Name} ({ID_Lat}°N, {ID_Lon}°E)"
    ax1.set_title(subTitle, fontsize=fontsize, x=0.03, y=0.8, 
                  loc='left', weight='normal', alpha=1, color='none')
    return None

def get_letter(index):
    """将索引转换为字母标签（A, B, C...）"""
    return chr(65 + index)

# 定义要分析的站点
ID_Names = ['J4901', 'J5157', 'J6350', 'I8882']

# 确保ID列为字符串类型
db_df['ID'] = db_df['ID'].astype(str)

# 创建图形
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(4.5*3, 2*3), sharey='all', dpi=200)
axs = axes.flatten()
fontsize = 13

# 初始化存储评分结果的列表
metrics_data = []

# 遍历每个站点
for ID_Index, ID_Name in enumerate(ID_Names):
    ID_df = db_df[db_df['ID'] == ID_Name].copy()
    
    # 移除可能存在缺失值的行
    valid_data = ID_df[['WRF-H', 'WRF-S', 'OBS_SM_10cm']].dropna()
    
    if len(valid_data) > 0:
        # 计算WRF-H的评分
        corr_H, rmse_H, bias_H = calculate_metrics(
            valid_data['WRF-H'].values, 
            valid_data['OBS_SM_10cm'].values
        )
        
        # 计算WRF-S的评分
        corr_S, rmse_S, bias_S = calculate_metrics(
            valid_data['WRF-S'].values, 
            valid_data['OBS_SM_10cm'].values
        )
        
        # 保存结果
        metrics_data.append({
            'ID': ID_Name,
            'WRF-H_TCORR': round(corr_H, 3),
            'WRF-H_TRMSE': round(rmse_H, 3),
            'WRF-H_TBIAS': round(bias_H, 3),
            'WRF-S_TCORR': round(corr_S, 3),
            'WRF-S_TRMSE': round(rmse_S, 3),
            'WRF-S_TBIAS': round(bias_S, 3)
        })
    
    # 绘制图形
    ax1 = axs[ID_Index]
    plot_SM_precipitation(ID_df, ID_Index, ID_Name, ax1)
    
    if ID_Index == 0:
        ax1.set_ylabel("JTB SM (m³/m³)", fontsize=fontsize)
    if ID_Index == 3:
        ax1.set_ylabel("SAM SM (m³/m³)", fontsize=fontsize)

# 消除多余的子图
for idx in range(len(ID_Names), len(axs)):
    fig.delaxes(axs[idx])

# 创建统一的图例
lines, labels = fig.axes[-1].get_legend_handles_labels()
fig.legend(lines, labels, 
           bbox_to_anchor=(0.65, 0.08), ncol=4, framealpha=1, edgecolor='none')

# 设置子图间距
plt.subplots_adjust(wspace=0.1, hspace=0.15)

# 创建评分DataFrame
if metrics_data:
    metrics_df = pd.DataFrame(metrics_data)
    
    # 设置ID为索引
    metrics_df.set_index('ID', inplace=True)
    
    # 可以选择重新排列列的顺序
    column_order = [
        'WRF-H_TCORR', 'WRF-H_TRMSE', 'WRF-H_TBIAS',
        'WRF-S_TCORR', 'WRF-S_TRMSE', 'WRF-S_TBIAS'
    ]
    metrics_df = metrics_df[column_order]
    
    print("土壤湿度模拟评分结果:")
    print("=" * 70)
    print(metrics_df.to_string())
    print("\n评分说明:")
    print("TCORR: 时间序列相关系数 (越接近1越好)")
    print("TRMSE: 时间序列均方根误差 (越接近0越好)")
    print("TBIAS: 时间序列偏差 (越接近0越好)")
    
    # 也可以保存到文件
    metrics_df.to_csv('soil_moisture_InStu_metrics.csv', float_format='%.3f')
else:
    print("未找到有效数据计算评分")

plt.show()